In [ ]:
import cr_autophagy as cra
from pathlib import Path

import importlib
importlib.reload(cra)
import glob
import numpy as np
import multiprocessing as mp
import tqdm
import matplotlib.pyplot as plt
import seaborn as sb

discretization_factor = 0.5
bw_method = 0.12

def gather_result(output_path):
    simulation_settings = cra.get_simulation_settings(output_path)
    max_iter = np.max(cra.get_all_iterations(output_path))

    kde_clrs = cra.get_clusters_kde(output_path, max_iter, discretization_factor, bw_method)
    if kde_clrs != None and kde_clrs.validate():
        kde_clusters = kde_clrs.n_clusters
        kde_clusters_at_cargo = len(kde_clrs.clusters_at_cargo(relative_radial_distance=0.5))
    else:
        kde_clusters = np.nan
        kde_clusters_at_cargo = np.nan
    
    graph_clrs = cra.get_clusters_graph(output_path, max_iter)
    if graph_clrs != None and graph_clrs.validate():
        graph_clusters = graph_clrs.n_clusters
        graph_clusters_at_cargo = len(graph_clrs.clusters_at_cargo(relative_radial_distance=0.5))
    else:
        graph_clusters = np.nan
        graph_clusters_at_cargo = np.nan

    return {
        "kde_clusters": kde_clusters,
        "kde_clusters_at_cargo": kde_clusters_at_cargo,
        "graph_clusters": graph_clusters,
        "graph_clusters_at_cargo": graph_clusters_at_cargo,
        "potential_strength_r11_r11": simulation_settings.potential_strength_r11_r11,
        "potential_strength_cargo_r11": simulation_settings.potential_strength_cargo_r11,
        "potential_strength_cargo_r11_avidity": simulation_settings.potential_strength_cargo_r11_avidity,
        "kb_temperature_r11": simulation_settings.kb_temperature_r11,
        "iteration": max_iter,
        "output_path": output_path,
    }


# args = [Path(folder) for folder in glob.glob("out/autophagy/explore*")]
# args = list(filter(lambda path: len(glob.glob(str(path) + "/cell_storage/json/*")) == 51, args))
# pool = mp.Pool(10)
# results = cra.pd.DataFrame(list(tqdm.tqdm(pool.imap_unordered(gather_result, args), total=len(args))))
# results.to_csv("parameter_space.csv")

In [ ]:
# Filter for NaN values
results = cra.pd.read_csv("parameter_space.csv")
df = results[results["kde_clusters"]>0]

# Get unique entries for each parameter
values_potential_strength_r11_r11 = np.unique(df["potential_strength_r11_r11"])
values_potential_strength_cargo_r11 = np.unique(df["potential_strength_cargo_r11"])
values_potential_strength_cargo_r11_avidity = np.unique(df["potential_strength_cargo_r11_avidity"])
values_kb_temperature_r11 = np.unique(df["kb_temperature_r11"])

print(values_potential_strength_r11_r11)
print(values_potential_strength_cargo_r11)
print(values_potential_strength_cargo_r11_avidity)
print(values_kb_temperature_r11)

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(12, 12))

fig.suptitle("KDE Clusters", fontsize=20)
sb.violinplot(x="kb_temperature_r11", y="kde_clusters_at_cargo", data=df, formatter=lambda x: f"{x:05.4f}", ax=ax[0,0])
sb.violinplot(x="potential_strength_cargo_r11_avidity", y="kde_clusters_at_cargo", data=df, formatter=lambda x: f"{x:05.4f}", ax=ax[0,1])
sb.violinplot(x="potential_strength_cargo_r11", y="kde_clusters_at_cargo", data=df, formatter=lambda x: f"{x:05.4f}", ax=ax[1,0])
sb.violinplot(x="potential_strength_r11_r11", y="kde_clusters_at_cargo", data=df, formatter=lambda x: f"{x:05.4f}", ax=ax[1,1])
fig.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sb
fig, ax = plt.subplots(2, 2, figsize=(12, 12))

fig.suptitle("Graph Clusters", fontsize=20)
sb.violinplot(x="kb_temperature_r11", y="graph_clusters_at_cargo", data=df, formatter=lambda x: f"{x:05.4f}", ax=ax[0,0])
sb.violinplot(x="potential_strength_cargo_r11_avidity", y="graph_clusters_at_cargo", data=df, formatter=lambda x: f"{x:05.4f}", ax=ax[0,1])
sb.violinplot(x="potential_strength_cargo_r11", y="graph_clusters_at_cargo", data=df, formatter=lambda x: f"{x:05.4f}", ax=ax[1,0])
sb.violinplot(x="potential_strength_r11_r11", y="graph_clusters_at_cargo", data=df, formatter=lambda x: f"{x:05.4f}", ax=ax[1,1])
fig.tight_layout()
plt.show()

In [ ]:
output_path = cra.get_last_output_path()
max_iter = max(cra.get_all_iterations(output_path))

In [ ]:
discretization_factor = 0.5
bw_method = 0.12
fig = cra.plot_cluster_distribution(output_path, max_iter, discretization_factor, bw_method)

In [ ]:
fig = cra.save_kernel_density(
    output_path,
    max_iter,
    overwrite=True,
    threshold=0.1,
    discretization_factor=0.5,
    bw_method=0.12
)

In [ ]:
# cra.save_all_kernel_density(output_path, threshold=0.6, overwrite=True, bw_method=0.4, discretization_factor=0.5)
# 
# bashcmd = f"ffmpeg -v quiet -stats -y -r 30 -f image2 -pattern_type glob -i '{output_path}/kernel_density/*.png' -c:v h264 -pix_fmt yuv420p -strict -2 {output_path}/kernel_density_movie.mp4"
# os.system(bashcmd)